In [8]:
import numpy as np
from tqdm import tqdm
import pickle
np.set_printoptions(threshold=np.nan)
import numpy as np
import os
import sys
cwd = os.getcwd()

In [9]:
#Loading the original datasets
x_train_full = np.load(os.path.join(cwd,'train/train_array.npy'))
y_train_full = np.load(os.path.join(cwd,'train/fold_stance_labels.npy'))
x_test_full = np.load(os.path.join(cwd,'dev/train_array.npy'))
y_test_full = np.load(os.path.join(cwd,'dev/fold_stance_labels.npy'))
ids_test_full = np.load(os.path.join(cwd,'dev/tweet_ids.npy'))
ids_train_full = np.load(os.path.join(cwd,'train/tweet_ids.npy'))

In [10]:
#preprocessing dev preprocessed data
def create_data_with_preceding_thread(x_data, y_data, ids):
    """
    the function takes as input preprocessed data obtained through running the preprocessing code 
    from a baseline suggested here: https://github.com/kochkinaelena/RumourEval2019 on the code shared 
    on the competition codalab webpage and returns the dataset
    where each comment is represented as an average of vectors of all preceding comments + self vector
    x: original preprocessed data, vectors
    y: original preprocessed gold labels, format: matrix where each row is a list of len(max depth of thread) with each item a prediction of the respective comment
    ids: original ids (tree structure)
    """
    
    id_list = [] #list of comment ids 
    print(id_list)
    vector_of_preceding_tree_list = [] #list of vectors of each comment, with the vector being the average of all the previous comments
    y_per_statement = [] #correct label for each comment 
    for tree in tqdm(ids.tolist()):
        #print(type(tree))
        idx_of_tree = ids.tolist().index(tree)
        #print(tree, idx_of_tree)
        for comment_id in tree:
            idx_of_comment_in_tree = tree.index(comment_id)
            if comment_id not in id_list:
                id_list.append(comment_id)
                comment_vector = np.zeros(len(x_data[idx_of_tree][idx_of_comment_in_tree]))
                for i in range(idx_of_comment_in_tree+1):
                    vector_to_add = x_data[idx_of_tree][i]
                    comment_vector = comment_vector + x_data[idx_of_tree][i]
                comment_vector = np.divide(comment_vector, idx_of_comment_in_tree+1)
                vector_of_preceding_tree_list.append(comment_vector)
                y_per_statement.append(y_data[idx_of_tree][idx_of_comment_in_tree])
    return [vector_of_preceding_tree_list, y_per_statement, id_list]

In [11]:
dev_x_preceding_thread, dev_y_preceding_thread, dev_id_preceding_thread = create_data_with_preceding_thread(x_test_full, y_test_full, ids_test_full)

100%|██████████| 966/966 [00:00<00:00, 14249.53it/s]

[]


In [12]:
# np.save("dev_x_preceding_thread", np.array(dev_x_preceding_thread))
# np.save("dev_y_preceding_thread", np.array(dev_y_preceding_thread))
# np.save("dev_id_preceding_thread", np.array(dev_id_preceding_thread))

In [13]:
train_x_preceding_thread, train_y_preceding_thread, train_id_preceding_thread = create_data_with_preceding_thread(x_train_full, y_train_full, ids_train_full)

 36%|███▋      | 1291/3560 [00:00<00:00, 12838.02it/s]

[]


100%|██████████| 3560/3560 [00:00<00:00, 6485.86it/s] 


In [14]:
# np.save("train_x_preceding_thread", np.array(train_x_preceding_thread))
# np.save("train_y_preceding_thread", np.array(train_y_preceding_thread))
# np.save("train_id_preceding_thread", np.array(train_id_preceding_thread))